# Big Cat Classification using TinyVgg

## Importing necessary Libraries

In [ ]:
import torch
import os
import numpy as np

import random
from PIL import Image
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
torch.__version__

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")

## PreProcessing Data

In [ ]:
DIR_PATH = 'tiger_dataset'

In [ ]:
for root, dirnames, files in os.walk(DIR_PATH):
    print(f"There are {len(dirnames)} directories and {len(files)} images in '{root}'.")

In [ ]:
#train directory
TRAIN_DIR = os.path.join(DIR_PATH,'train')

#test directory
TEST_DIR = os.path.join(DIR_PATH,'test')

#val directory
VAL_DIR = os.path.join(DIR_PATH,'valid')

TRAIN_DIR, TEST_DIR, VAL_DIR

In [ ]:
#creating a list of all images
img_list = list()

for root, dirname, files in os.walk(DIR_PATH):
    for file in files:
        if file.endswith(".jpg"):
            img_list.append((os.path.join(root, file)))

In [ ]:
img_list[:5]

###  Random Image with size

In [ ]:
random.seed(42)

random_path = random.choice(img_list)

image = Image.open(random_path)
label = os.path.dirname(random_path)
label = os.path.basename(label)

print(f'Height of Image| {image.height}')
print(f'Width of Image| {image.width}')
print(f'Path of Image| {random_path}')
print(f'Class of Image| {label}')
image

In [ ]:
random.seed(42)

random_list = []
random_class = []

for i in range(4):
    p = random.choice(img_list)
    random_list.append(p)
    name = os.path.dirname(p)
    name = os.path.basename(name)
    random_class.append(name)

In [ ]:
random_list,random_class

In [ ]:
#Random Images and their size
for num in range(4):
    with Image.open(random_list[num]) as f:
        plt.figure(figsize = (4,4))
        plt.imshow(f) 
        plt.title(f"Class: {random_class[num]}\nSize: {f.size}")
        plt.axis("off")

In [ ]:
#transform function for prepairing the images for neural network
data_transform = transforms.Compose([
    #resize the image
    transforms.Resize(size=(64, 64)),
    #Horizontal flip with .5 probability
    transforms.RandomHorizontalFlip(p=0.5),
    #convert to Tensor
    transforms.ToTensor() 
])

In [ ]:
#creation of Train Data and test data
train_data = datasets.ImageFolder(root=TRAIN_DIR,
                                  transform=data_transform, 
                                  target_transform=None)

test_data = datasets.ImageFolder(root=TEST_DIR,
                                transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

### Before and after transform

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=2):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f) 
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {os.path.basename(os.path.dirname(image_path))}", fontsize=16)

plot_transformed_images(img_list, 
                        transform=data_transform, 
                        n=3)

In [ ]:
#all the classes that can be determined
class_names = train_data.classes
class_names

In [ ]:
#Iterable dataloaders to Create Batches of data to load into the model
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=32, # how many samples per batch?
                              num_workers=os.cpu_count()-1,# how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=1, 
                             num_workers=os.cpu_count()-1, 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

In [ ]:
img, label = next(iter(train_dataloader))

#current image shape
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

In [ ]:
def display_random_images(dataset: torch.utils.data.dataset.Dataset,
                          classes: [] = None,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None):
    
    # Adjust display if n too high or else images will overlap
    if n > 7:
        n = 7
        display_shape = False
        print("For display purposes, n shouldn't be larger than 6, setting to 6 and removing shape display.")
    
    #Set random seed to create reproducibility
    if seed:
        random.seed(seed)

    #Get random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # Setup plot
    plt.figure(figsize=(16, 10))

    #Loop through samples and display random samples 
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

        #Adjust image tensor for plotting: [color_channels, height, width(pytoch)] -> [color_channels, height, width(matplotlib)]
        targ_image_adjust = targ_image.permute(1, 2, 0)

        # Plot adjusted samples
        plt.subplot(1, n, i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        if classes:
            title = f"class: {classes[targ_label]}"
        plt.title(title)

In [ ]:
display_random_images(train_data, 
                      n=7, 
                      classes=class_names,
                      seed=42)

## Model Creation

In [ ]:
#Import the TinyVGG model 
from TinyVGG import tinyVGG

model1 = tinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, #intermediate units
                  output_shape=len(train_data.classes)).to(device) #output shape 10 in this case

In [ ]:
model1

###  Model summary

In [ ]:
#How each layer accepts and outputs the image
from torchinfo import summary
summary(model1, input_size=[1, 3, 64, 64])

## Train and test loops

In [ ]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward Tracking(autograd)
        loss.backward()

        # 5. Optimizer step(making changes to the weigths)
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [ ]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
    
    # Turn on inference context manager this remove grad
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)
    
            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [ ]:
#Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = torch.nn.CrossEntropyLoss(),
          epochs: int = 20):
    
    #Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    #Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model1,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        #Print Results after each epoch
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        #Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

## Training of the model 

In [ ]:
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 1

# Setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model1.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model
model_results = train(model=model1, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)

# End the timer and duration of training
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
model_results.keys()

## Loss and accuracy

In [ ]:
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(model_results)

In [ ]:
val_img_list = list()

for root, dirname, files in os.walk(VAL_DIR):
    for file in files:
        if file.endswith(".jpg"):
            val_img_list.append((os.path.join(root, file)))

In [ ]:
val_img_list[:5]

In [ ]:
def predict_img_list(seed=42,
                k=5):
    random.seed(seed)
    
    val_random_path = random.choice(val_img_list)
        
    return val_random_path

In [ ]:
val_random_path = predict_img_list(seed=42,
                              k=5)
val_random_path

## Prediction on random Image

In [ ]:
import torchvision

#read the image and convert that to tensor
custom_image = Image.open(val_random_path)
custom_image_resize = custom_image.resize((64,64),resample=0)
trans = transforms.functional.pil_to_tensor(custom_image_resize).type(torch.float)
 
    
model1.eval()
with torch.inference_mode():
    val_image_pred = model1(trans.unsqueeze(dim=0).to(device))

In [ ]:
val_image_pred

In [ ]:
val_image_pred_probs = torch.softmax(val_image_pred,dim=1)
    
    
print(f"Prediction probabilities: {val_image_pred_probs}")

In [ ]:
# Convert prediction probabilities -> prediction labels
val_image_pred_label= torch.argmax(val_image_pred_probs, dim=1)
    
    
print(f"Prediction label: {val_image_pred_label}")

In [ ]:
val_image_pred_class=class_names[val_image_pred_label.cpu()] # put pred label to CPU, otherwise will error
val_image_pred_class

In [ ]:
def pred_and_plot_image(model, 
                        image_path, 
                        class_names,
                        device):
    """Makes a prediction on a target image and plots the image with its prediction."""
    
    # 1. Load in image
    target_image = Image.open(image_path)
    
    # 2. Resizing the image
    custom_image_resize = custom_image.resize((64,64),resample=0)
    
    # 3. Convert the image to tensor
    trans = transforms.functional.pil_to_tensor(custom_image_resize).type(torch.float)
    
    # 4. Make sure the model is on the target device
    model.to(device)
    
    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # Add an extra dimension to the image
        target_image = target_image.unsqueeze(dim=0)
    
        # Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(target_image.to(device))
        
    # 6. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 7. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
    
    # 8. Plot the image alongside the prediction and prediction probability
    plt.imshow(target_image.squeeze().permute(1, 2, 0)) # make sure it's the right size for matplotlib
    img = np.asarray(Image.open(image_path))
    plt.imshow(img)
    if class_names:
        title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    else: 
        title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    plt.title(title)
    plt.axis(False);

In [ ]:
pred_and_plot_image(model=model1,
                    image_path=val_random_path,
                    class_names=class_names,
                    device=device)

## Validation Accuracy

In [ ]:
valid_data = datasets.ImageFolder(root=VAL_DIR,
                                transform=data_transform)

valid_dataloader = DataLoader(dataset=valid_data, 
                             batch_size=5, 
                             num_workers=1, 
                             shuffle=False)

In [ ]:
def evaluate(model, dataloader):
    model1.eval()
    with torch.inference_mode():
        total, correct = 0, 0
        for data in dataloader:
            # Get the inputs and move them to the device
            img, label = next(iter(dataloader))
            img, label = img.to(device), label.to(device)

            # Forward pass
            outputs = model(img)
            predicted = torch.softmax(outputs,dim=1)
            predicted = torch.argmax(outputs, dim=1)

            # Record the accuracy
            total += label.size(0)
            correct += (predicted == label).sum().item()

    # Print the accuracy
    print(f'Accuracy of the model on the {total} images: {100*correct/total} %')

In [ ]:
print('Valid Case')
# Evaluate valid data

evaluate(model1, valid_dataloader)

## Saving the model

In [ ]:
torch.save(model1.state_dict(),'Best_model/tinyvgg/model.pth')